In [73]:
import pandas as pd
import re # Regex Operations
from pathlib import Path
import helpers.project_config as cfg

## General Helper Functions

In [74]:
def _read_dataset(path: str | Path) -> pd.DataFrame:
    path = Path(path)
    suffix = path.suffix.lower()

    if suffix == ".csv":
        return pd.read_csv(path)

    raise ValueError(f"Unsupported input format: {suffix} for {path} \n Use .csv")


def _write_dataset(df: pd.DataFrame, path: str | Path) -> None:
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    suffix = path.suffix.lower()

    if suffix == ".csv":
        df.to_csv(path, index=False)
        return

    raise ValueError(f"Unsupported output format: {suffix} for {path}")


## Transformation
contains reusable transformation functions

In [75]:
def normalize_dataframe_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize DataFrame column names by:
      - replacing whitespace with underscores
      - converting uppercase letters to lowercase

    Whitespace runs (e.g., multiple spaces, tabs) are collapsed into a single "_".

    Args:
        df: Input pandas DataFrame.

    Returns:
        The DataFrame with normalized column names.

    Raises:
        TypeError: If df is not a pandas DataFrame.
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"df must be a pandas DataFrame, got {type(df)!r}")

    _WHITESPACE_RE = re.compile(r"\s+")
    # \s means “any whitespace character” (space, tab, newline, etc.)
    # + means “one or more of the previous character”

    def _normalize(name) -> str:
        text = str(name).strip()
        text = _WHITESPACE_RE.sub("_", text)
        return text.lower()

    new_columns = [_normalize(col) for col in df.columns]
    return df.rename(columns=dict(zip(df.columns, new_columns)))

### Smoke Tests

In [76]:
# Smoke tests for normalize_dataframe_columns() 
def run_normalize_dataframe_columns_tests() -> None:
    # Basic case
    df = pd.DataFrame(columns=["First Name", "Last Name"])
    out = normalize_dataframe_columns(df)
    assert list(out.columns) == ["first_name", "last_name"]

    # Edge case 1: multiple whitespace types collapse to single underscore (space/tab/newline)
    df = pd.DataFrame(columns=["A   B", "C\tD", "E\nF", "G \t \n H"])
    out = normalize_dataframe_columns(df)
    assert list(out.columns) == ["a_b", "c_d", "e_f", "g_h"]

    # Edge case 2: non-string column names (should not crash; should stringify)
    df = pd.DataFrame(columns=[1, ("A", "B"), 3.14])
    out = normalize_dataframe_columns(df)
    assert list(out.columns) == ["1", "('a',_'b')", "3.14"]

    # Edge case 3: leading/trailing whitespace gets stripped before replacement
    df = pd.DataFrame(columns=["  Hello World  ", "\tFoo Bar\t"])
    out = normalize_dataframe_columns(df)
    assert list(out.columns) == ["hello_world", "foo_bar"]

    # Already-normalized should stay normalized
    df = pd.DataFrame(columns=["already_ok", "ok2"])
    out = normalize_dataframe_columns(df)
    assert list(out.columns) == ["already_ok", "ok2"]

    # Type error on wrong input
    try:
        normalize_dataframe_columns(["not", "a", "df"])  
        assert False, "Expected TypeError for non-DataFrame input"
    except TypeError:
        pass

    print("✅ normalize_dataframe_columns smoke tests passed")

def basic_sanity_checks() -> None:
    # TERM_SIZES attribute
    if not hasattr(cfg, "TERM_SIZES"):
        raise AttributeError("cfg.TERM_SIZES is missing in helpers/project_config.py")

    term_sizes = list(cfg.TERM_SIZES)
    if not term_sizes:
        raise ValueError("cfg.TERM_SIZES is empty")

run_normalize_dataframe_columns_tests()
basic_sanity_checks()

✅ normalize_dataframe_columns smoke tests passed


## Pipeline
orchestration (read -> transform -> write)

### Project level dataset fix (adding term, comment density)

**Specifications**


Problem: 
`output/interim_results/aggregated_metrics_v4_061025.csv` has no term column
`output/interim_results/cleaned_aggregated_metrics_v4_061025.csv` has not all projects anymore

Goal: file with all projects and term column stored under `project_level_dataset_path_raw`

Process: add term column (integer 1, 2, 3 or 4) to `/output/interim_results/aggregated_metrics_v4_061025.csv` based on `TERM_SIZES` list from `project_config.py` and store it to new path. Sum of integers in `TERM_SIZES` should equal the number of projects

In [77]:
def add_term_column_to_project_dataset() -> None:
    """
    Adds a 'term' column to the project-level dataset based on TERM_SIZES in config.
    Validates that the number of rows matches the sum of TERM_SIZES.
    Writes the updated dataset to the configured raw project-level dataset path.
    """
    # Inputs / outputs
    src_path = Path("output/interim_results/aggregated_metrics_v4_061025.csv")
    dst_path = Path(cfg.project_level_dataset_path_raw)

    project_df = _read_dataset(src_path)

    # Checks
    term_sizes = cfg.TERM_SIZES
    n_rows = len(project_df)

    expected_total = sum(int(x) for x in term_sizes)
    if expected_total != n_rows:
        raise ValueError(
            "TERM_SIZES total does not match number of rows in the dataset.\n"
            f"sum(TERM_SIZES)={expected_total}, rows={n_rows}\n"
            "Fix TERM_SIZES or verify the input file has exactly one row per project."
        )

    if project_df['project index'].nunique() != n_rows:
        print(
            "⚠️ Warning: Duplicated projects (Project Index is not unique to each entry)"
        )

    # Add term column based on TERM_SIZES
    # term_values: list[int] = []
    # for term_idx, size in enumerate(term_sizes, start=1):
    #     term_values.extend([term_idx] * int(size))

    term_values = pd.Series(
        [i for i, size in enumerate(term_sizes, start=1) for _ in range(size)]
    )

    project_df["term"] = term_values

    _write_dataset(project_df, dst_path)

    # Report
    counts = project_df["term"].value_counts().sort_index()
    print(f"✅ Wrote project-level raw dataset with term column to: {dst_path}")
    print(f"Rows: {n_rows} | Term distribution: {counts.to_dict()}")

# add_term_column_to_project_dataset()

In [ ]:
src = cfg.project_level_dataset_path_raw
def add_comment_density(src: str) -> None:
    df = _read_dataset(src)
    row_in = df.shape
    df['comment_density'] = df['3_folders_total_lines_of_comments'] / df['3_folders_total_lines']
    print(f"From {row_in} to {df.shape}")
    _write_dataset(df, src)

# add_comment_density(src)

From (669, 103) to (669, 104)


### Normalization Column Names

- replacing whitespace with underscores
- converting uppercase letters to lowercase

In [78]:
def show_columns(df: pd.DataFrame) -> list[str]:
    return df.columns.tolist()


# --- Project-level ---
def normalize_project_df():
    project_df_raw = _read_dataset(cfg.project_level_dataset_path_raw)
    project_df_norm = normalize_dataframe_columns(project_df_raw)
    _write_dataset(project_df_norm, cfg.project_level_dataset_path_normalized)
    print(
        f"project_level: {project_df_raw.shape} -> {project_df_norm.shape} "
        f"written to {cfg.project_level_dataset_path_normalized}"
    )
    print(f"columns before: {show_columns(project_df_raw)}")
    print(f"columns after: {show_columns(project_df_norm)}")


# --- Commit-level ---
def normalize_commit_df():
    commit_df_raw = _read_dataset(cfg.commit_level_dataset_path_raw)
    commit_df_norm = normalize_dataframe_columns(commit_df_raw)
    _write_dataset(commit_df_norm, cfg.commit_level_dataset_path_normalized)
    print(
        f"commit_level: {commit_df_raw.shape} -> {commit_df_norm.shape} "
        f"written to {cfg.commit_level_dataset_path_normalized}"
    )
    print(f"columns before: {show_columns(commit_df_raw)}")
    print(f"columns after: {show_columns(commit_df_norm)}")


# run only once whenever raw file changes
# normalize_project_df()
# normalize_commit_df()


### Commit level dataset fix (drop file_count, rename dmm column)

drop file_count as it is the same as files_changed

In [ ]:
def drop_doubled_col():
    df = _read_dataset(cfg.commit_level_dataset_path_normalized)
    # print(df.columns)
    if 'file_count' in df.columns:
        df = df.drop(columns=['file_count'])
        print("dropped it")
    else:
        print("column not in dataframe anymore")
    # print(df.columns)
    _write_dataset(df, cfg.commit_level_dataset_path_normalized)

# drop_doubled_col()

column not in dataframe anymore


In [ ]:
path = cfg.commit_level_dataset_path_cleaned
old_names = ['unit_size_(dmm)', 'complexity_(dmm)', 'interface_(dmm)']
new_names = ['unit_size_dmm', 'complexity_dmm', 'interface_dmm']

def rename_dmm_cols(path: str) -> None:
    df = _read_dataset(path)
    has_old = all(name in df.columns for name in old_names)
    has_new = all(name in df.columns for name in new_names)
    if has_old:
        df = df.rename(columns={old_names[0]: new_names[0], old_names[1]: new_names[1], old_names[2]: new_names[2]})
        _write_dataset(df=df, path=path)
        print("successfully renamed")
    elif has_new:
        print("already has new names")
    else:
        print(f"something is wrong - {df.columns}")
    

# rename_dmm_cols(path)



already has new names


### Add Dev Column to Commit Level DF

Meaning of Dev Column:
| 'Dev' | Meaning |
|----------|----------|
| -1    | Bot  |
| 0    | Not assignable  |
| 1    | Dev 1 of project |
| 2    | Dev 2 of project |
| 3    | Dev 3 of project  |
| ...    | ...  |

optimal is if a project contains only bots and two developers

In [79]:
# drop column dev for testing add_dev_column_to_commit_df()
def drop_dev_column_commit_df() -> None:
    df_commits = _read_dataset(cfg.commit_level_dataset_path_normalized)
    if "dev" in df_commits.columns:
        df_commits = df_commits.drop(columns=["dev"])
        _write_dataset(df_commits, cfg.commit_level_dataset_path_normalized)
        print(f"Dropped 'dev' column from {cfg.commit_level_dataset_path_normalized}")
    else:
        print(f"No 'dev' column found in {cfg.commit_level_dataset_path_normalized}")

# drop_dev_column_commit_df()

In [80]:
def add_dev_column_to_commit_df()-> None:
    # Import mapping df with dev column
    filename = "dev_mapping_manually.csv"
    dataset_path = cfg.base_path + "output/interim_results/" + filename
    df_mapping = _read_dataset(dataset_path)
    df_mapping = normalize_dataframe_columns(df_mapping)
    
    if "dev" not in df_mapping.columns: # check correctness of file
        raise ValueError(
            "Mapping file must contain a 'dev' column after normalization "
            f"(found: {list(df_mapping.columns)})"
        )

    if len(df_mapping[df_mapping['dev'].isna()]) != 0: # check correctness of file
        raise ValueError(
            "Mapping file must contain a 'dev' value for each entry, some dev values are missing "
            f"(found: {list(df_mapping[df_mapping['dev'].isna()])})"
        )


    # Import commit dataframe
    df_commits = _read_dataset(cfg.commit_level_dataset_path_normalized)

    if "dev" in df_commits.columns: # check correctness of file
        raise ValueError(
            "Mapping file contains a 'dev' column indicates values were already added"
            f"(found: {list(df_commits.columns)})"
        )

    # For stats later
    n_rows_in = len(df_commits)
    n_col_in = df_commits.shape[1]
    cols_in = set(df_commits.columns)

    # Calculate bot list
    # check which unique combination of (author name, author email) commited to more than one project
    dev_projects = (
        df_commits.dropna(subset=['project', 'author_name', 'author_email'])
        .drop_duplicates(subset=['project', 'author_name', 'author_email'])  # one per project
        .groupby(['author_name', 'author_email'])['project']
        .nunique()
        .reset_index(name='n_projects')
    )

    multi_project_devs = dev_projects[dev_projects['n_projects'] > 1].sort_values(
        'n_projects', ascending=False
    )

    bots = multi_project_devs[multi_project_devs['n_projects'] > 3]
    
    # print(bots[['author_name', 'author_email']])

    # bots don't need a project column, their dev column is equal to -1
    bot_mapping = bots.copy()
    bot_mapping = bot_mapping[['author_name', 'author_email']]
    bot_mapping['dev'] = -1

    # Extend commit dataframe with dev column from bot by matching on mail and name
    df_commits = df_commits.merge(
        bot_mapping,
        on=['author_name', 'author_email'],
        how='left',
        validate="many_to_one",
    )
    
    # Extend commit dataframe with dev column by matching on project, mail and name
    mapping_required = {"project", "author_name", "author_email", "dev"}
    mapping_missing = mapping_required - set(df_mapping.columns) 
    if mapping_missing: # check that all required columns are present
        raise ValueError(
            "Mapping dataset is missing required columns: "
            f"{sorted(mapping_missing)} (found: {list(df_mapping.columns)})"
        )

    df_mapping = df_mapping.rename(columns={"dev": "dev_map"})

    df_commits = df_commits.merge( # creates dev_map column additional to dev column from bot
        df_mapping,
        on=['project', 'author_name', 'author_email'],
        how="left",
        validate="many_to_one",
    )

    df_mapping = df_mapping.rename(columns={"dev_map": "dev"})

    df_commits["dev"] = df_commits["dev"].combine_first(df_commits["dev_map"]) # merges dev and dev_map column, prioritize dev column from bot mapping
    df_commits.drop(columns=["dev_map"], inplace=True)

    if len(df_commits[df_commits['dev'].isna()]) != 0: # check correctness of df
        raise ValueError(
            "After merging dataframes dev column must contain a value for each entry, some dev values are missing "
            f"(found: {list(df_commits[df_commits['dev'].isna()])})"
    )



    # Stats
    n_rows_out = len(df_commits)
    n_col_out = df_commits.shape[1]
    new_columns = set(df_commits.columns) - cols_in
    n_dev_missing = int(df_commits["dev"].isna().sum())
    n_bots = len(bot_mapping)
    dev_counts = (
        df_commits["dev"].fillna("NA").value_counts(dropna=False).to_dict()    
        if "dev" in df_commits.columns
        else {}
    )

    print(
        "✅ Added 'dev' column to commit-level dataset\n"
        f"Rows: {n_rows_in} -> {n_rows_out}\n"
        f"Cols: {n_col_in} -> {n_col_out}\n"
        f"New columns: {new_columns}\n"
        f"Missing dev: {n_dev_missing}\n"
        f"Bots flagged (dev=0): {n_bots}\n"
        f"Dev distribution: {dev_counts}\n"
    )

    # Store extended commit dataframe 
    _write_dataset(df_commits, cfg.commit_level_dataset_path_normalized)

# run only once whenever raw file changes, check writting is enabled at bottom of function
# add_dev_column_to_commit_df()


Check which commit entries have 'dev' == -1, list all bots with how many commits they made

In [81]:
df_commits = _read_dataset(cfg.commit_level_dataset_path_normalized)
bot_list = df_commits[df_commits['dev'] == -1]
bot_list[['author_name', 'author_email']].drop_duplicates()
bot_list[['author_name', 'author_email']].value_counts()

author_name       author_email                     
autobot           autobot@students.cs.ubc.ca           887
classy            classy@cs.ubc.ca                     486
310-bot           autobot@students.cs.ubc.ca           457
cs-310            cs-310-noreply@students.cs.ubc.ca    294
CPSC 310 autobot  310-bot@students.cs.ubc.ca           219
Service           autobot@students.cs.ubc.ca            51
Name: count, dtype: int64

### Clean Dataframes

#### Commit Level

In [92]:
df = _read_dataset(cfg.commit_level_dataset_path_normalized)
df[df['project'] == 14].shape 

(81, 20)

In [83]:
# remove merge commit
print("Number of projects BEFORE removing incompletes: ", df["project"].nunique())
print("Number of commits BEFORE quantile filtering: ", df.shape[0])
df = df[df['merge_commit'] == False]
print("Number of projects AFTER removing incompletes: ", df["project"].nunique())
print("Number of commits AFTER quantile filtering: ", df.shape[0])


Number of projects BEFORE removing incompletes:  681
Number of commits BEFORE quantile filtering:  86483
Number of projects AFTER removing incompletes:  681
Number of commits AFTER quantile filtering:  57744


In [84]:
# remove projects with only one commit
print("Number of projects BEFORE removing incompletes: ", df["project"].nunique())
print("Number of commits BEFORE quantile filtering: ", df.shape[0])
df = df[df.groupby("project")["project"].transform("size") > 1]
print("Number of projects AFTER removing incompletes: ", df["project"].nunique())
print("Number of commits AFTER quantile filtering: ", df.shape[0])

Number of projects BEFORE removing incompletes:  681
Number of commits BEFORE quantile filtering:  57744
Number of projects AFTER removing incompletes:  673
Number of commits AFTER quantile filtering:  57736


In [85]:
# remove projects with doubled hash
# Remove all duplicates
print("\nNumber of commits BEFORE removing duplicates: ", df.shape[0])
print("Number of projects BEFORE removing incompletes: ", df["project"].nunique())
# Identify and remove projects involved in shared commits
shared_projects = (
    df.loc[df['commit_hash'].duplicated(keep=False), 'project']
    .unique()
)

df = df[~df['project'].isin(shared_projects)]

print(f"Removed {len(shared_projects)} projects with shared commits:\n{shared_projects}")
print("Number of commits AFTER removing duplicates: ", df.shape[0])
print("Number of projects AFTER removing incompletes: ", df["project"].nunique())


Number of commits BEFORE removing duplicates:  57736
Number of projects BEFORE removing incompletes:  673
Removed 4 projects with shared commits:
[317 433 479 480]
Number of commits AFTER removing duplicates:  57372
Number of projects AFTER removing incompletes:  669


In [ ]:
# _write_dataset(df, cfg.commit_level_dataset_path_cleaned)

#### Project Level

In [87]:
df = _read_dataset(cfg.project_level_dataset_path_normalized)


In [88]:
df.columns

Index(['project_index', 'folder_count', 'file_count', 'file_count_tslike',
       'file_count_.ts', 'file_count_.tsx', 'file_count_.js',
       'file_count_.jsx', 'commit_count', 'unique_commit_authors',
       ...
       'typescript_total_lines_of_comments', 'typescript_blank_lines',
       'javascript_file_count', 'javascript_total_lines', 'javascript_sloc',
       'javascript_total_lines_of_comments', 'javascript_blank_lines',
       'cc_sum', 'cc_sum_3folders', 'term'],
      dtype='object', length=103)

In [89]:
print("Number of projects BEFORE removing incompletes: ", df["project_index"].nunique())
df = df[~df['project_index'].isin(shared_projects)]
print("Number of projects AFTER removing incompletes: ", df["project_index"].nunique())

Number of projects BEFORE removing incompletes:  681
Number of projects AFTER removing incompletes:  677


In [90]:
# remove projects with only one commit
print("Number of projects BEFORE removing incompletes: ", df["project_index"].nunique())

df = df[df["commit_count"] > 1]

print("Number of projects AFTER removing incompletes: ", df["project_index"].nunique())
df.shape

Number of projects BEFORE removing incompletes:  677
Number of projects AFTER removing incompletes:  669


(669, 103)

In [ ]:
# _write_dataset(df, cfg.project_level_dataset_path_cleaned)

In [93]:
df.columns

Index(['genai_period', 'term', 'project', 'author_name', 'author_email',
       'date', 'repository', 'insertions', 'deletions', 'total_lines',
       'files_changed', 'diff_lines', 'file_count', 'unit_size_(dmm)',
       'complexity_(dmm)', 'interface_(dmm)', 'commit_hash', 'merge_commit',
       'default_branch', 'dev'],
      dtype='object')

In [98]:
df['files_changed'].value_counts()

files_changed
1       25803
2       14443
3       10419
4        6479
5        4675
        ...  
1029        1
203         1
256         1
6026        1
5993        1
Name: count, Length: 330, dtype: int64

In [99]:
df['file_count'].value_counts()

file_count
1       25803
2       14443
3       10419
4        6479
5        4675
        ...  
1029        1
203         1
256         1
6026        1
5993        1
Name: count, Length: 330, dtype: int64